4.Server and Client based TCP1
=======
---
### 1. TCP/IP Protocol Layer
### 2. Procedure of TCP Server/Client
### 3. Iterative Server
---
#### 1. TCP/IP Protocol Layer
Definition.4.1.1. TCP/IP Protocol Layer :
Layer that have purpose of internet based data transmitting.

- The result of dividing a big problem into hierarchical problems.
- There are four layers in data transmitting procedure and there are seven layers in data transmitting procedure detailed.
- It can standardize and specialize the area for each layer.

4.1.1. layer flow of TCP and UDP socket
```c
sock = socket(PF_INET, SOCK_STREAM, IPPROTO_TCP);
```
```mermaid
graph LR
A[Link layer] --> B[IP layer]
B --> C[TCP layer]
B -- Not this way --> D[UDP layer]
C --> E[Application layer]
D -- Not this way --> E
```

```c
sock = socket(PF_INET, SOCK_DGRAM, IPPROTO_TCP);
```
```mermaid
graph LR
A[Link layer] --> B[IP layer]
B -- Not this way --> C[TCP layer]
B --> D[UDP layer]
C -- Not this way --> E[Application layer]
D --> E
```
4.1.2. Link and IP layer
1. Feature and role of Link layer
	- Result of standardization of physical area.
	- The area that contains physical standard network protocols like LAN, WAN, and MAN.
![enter image description here](https://media.geeksforgeeks.org/wp-content/uploads/Packet_flow_1.jpg)

2. Feature and role of IP layer
	- IP means Internet protocol.
	- It is a protocol about  path setting
	- There are no solution about error in this area.
	
4.1.3. TCP/UDP layer
3. Feature and role of TCP/UDP layer
	-  layer related to sending and receiving real data
	- It is also called the transport layer
	- TCP can be guarantee about data order and data transmitting. (Reliable protocol)
	- Since TCP is reliable, it is a complex protocol than UDP.
	- TCP contains a procedure of checking data.

4.1.4. Application Layer
1. Application Layer that is created by programmer
	- The application protocol consists of this layer.
	- This protocol is based on socket.
#### 2. Procedure of TCP Server/Client
4.2.1. Procedure
> socket() // create socket
> bind() // assign socket address
> listen() // wait for connection
> accept() // accept connection
> read()/write() // transmit the data
> close() // terminate the connection

4.2.2. Waiting for connection
```c
#include <sys/type.h>
int listen(int sock, int backlog);
```
- Success : 0
- Failure : -1
- sock : socket descriptor that ready to connection. (server socket)
- backlog : size of queue about waiting connection.

4.2.3. socket.h
1. How to find this file in linux system
```shell
$ find /usr/include -name socket.h

/usr/include/linux/socket.h
/usr/include/asm-generic/socket.h
/usr/include/x86_64-linux-gnu/bits/socket.h
/usr/include/x86_64-linux-gnu/asm/socket.h
/usr/include/x86_64-linux-gnu/sys/socket.h
```
2. Check the backlog value of listen()
- It is defined in "/usr/include/x86_64-linux-gnu/bits/socket.h"
	- #define SOMAXCONN = 128
```shell
$ sudo /sbin/sysctl -a | grep somaxconn
[sudo] password for heesung yang: 
net.core.somaxconn = 128 
sysctl: reading key "net.ipv6.conf.all.stable_secret" 
sysctl: reading key "net.ipv6.conf.default.stable_secret" 
sysctl: reading key "net.ipv6.conf.eno1.stable_secret" 
sysctl: reading key "net.ipv6.conf.lo.stable_secret" 
sysctl: reading key "net.ipv6.conf.wlx00089ff4f91e.stable_secret"
```

The following are part of the file socket.h.
```c
/* Protocol families */
#define PF_UNSPEC	0	/* Unspecified. */
#define PF_LOCAL	1	/* Local to host (pipes and file-domain). */
#define PF_UNIX		PF_LOCAL	/* POSIX name for PF_LOCAL. */
#define PF_FILE		PF_LOCAL	/* Another non-standard name for PF_LOCAL. */
#define PF_INET		2	/* IP protocol family. */
#define PF_AX25		3	/* Amateur Radio AX.25. */
#define PF_IPX		4	/* Novell Internet Protocol. */
#define	PF_APPLETALK	5	/* Appletalk DDP */
```
```c
/* Address families. */
#define AF_UNSPEC	PF_UNSPEC
#define AF_LOCAL	PF_LOCAL
#define AF_UNIX		PF_UNIX
#define AF_FILE		PF_FILE
#define AF_INET		PF_INET
#define AF_AX25		PF_AX25
#define AF_IPX		PF_IPX
```
```c
/* Socket level values. Others are defined in the appropriate headers.
	XXX These definitions also should go into the appropriate headers as far as they are available. */
#define SOL_RAW		255
#define SOL_DECENT	261
#define SOL_X25		262
#define SOL_PACKET	263
#define SOL_ATM		264	/* ATM layer (cell level). */
#define SOL_AAL		265	/* ATM Adaption Layer (packet level). */
#define SOL_IRDA	266

/* Maximum queue length specifiable by listen. */
#define SOMAXCONN	128
```

4.2.4. Accepting connection call of client
```c
#include <sys/socket.h>
int accept(int sock, struct sockaddr * addr, socklen_t * addlen);
```
- Success : socket  file descriptor
- Failure : -1
- sock : server socket descriptor
- addr : reference of socket address structure
- addlen : size of socket address structure

```c
#include <sys/socket.h>
int connect(int sock, const struct sockaddr * servaddr, socklen_t addrlen);
```
- Success : server socket descriptor
- Failure : -1
- sock : client socket descriptor
- servaddr : reference of client socket address structure
- addlen : size of socket address structure

Clients create the socket, and this socket call the funtion connect(). When the function connect() is called(), the server address is also sended.

#### 3. Iterative Server
Echo Server :  Retransmit the string that the client sends

```mermaid
graph LR
A[socket] --> B[bind]
B --> C[listen]
C --> D[accept]
D --> E[read/write]
E --> F[close client]
F --> G[close server]
F --> D
```
######  Figure 4.3.1

If you repeatedly call the accept function as shown in the figure 4.3.1, you can continue to accept the client's connection request. However, it is not a model that can provide services to more than one client at the same time.

```c
for ( i = 0 ; i < 5 ; i ++ ) {
	clnt_sock = accept(serv_sock, (struct sockaddr*)&clnt_adr, &clnt_adr_sz);
	if (clnt_sock == -1)
		error_handling("accept() error");
	else
		printf("Connected client %d \n", i+1);
	
	while ((str_len = read(clnt_sock, message, BUF_SIZE))!=0)
		write(clnt_sock, message, str_len);

	close(clnt_sock);
}
```
######  Figure 4.3.2 : Part of Sever code
It can connect with a client 5 times.

```c
while ( TRUE ) {
	fputs("Input message(Q to quit): ", stdout);
	fgets(message, BUF_SIZE, stdin);

	if (!strcmp(message, "q\n") || !strcmp(message, "Q\n"))
		break;
	write(sock, message, strlen(message));
	str_len = read(sock, message, BUF_SIZE-1);
	message[str_len] = NULL;
	printf("Message from server : %s",messsage);
}
```
######  Figure 4.3.3 :  Part of Client code
Until client get 'Q' or 'q', it can send the message to server repeatedly.

__CHECK__
